In [14]:
import pandas as pd 
import numpy as np

TWEET_DATA = pd.read_csv("dataset_final_fixing.csv", encoding = "ISO-8859-1")
TWEET_DATA['tweet'].str.encode('ascii', 'ignore')
TWEET_DATA.head()

,tweet,label
0,@LisaAmartatara3 @fadlizon Yg dibangga banggai...,1
1,RT @RustamIbrahim: POLLING: Dari 4 gubernur di...,1
2,ojok ampe Anies Baswedan menikmati hsil proyek...,1
3,RT @ghanieierfan: Pertemuan sangat mengharukan...,0
4,RT @SebastianTeza: jangn nympek Anies Baswedan...,1


In [15]:
# ------ Case Folding --------
# gunakan fungsi Series.str.lower() pada Pandas
TWEET_DATA['tweet'] = TWEET_DATA['tweet'].str.lower()


print('Case Folding Result : \n')
print(TWEET_DATA['tweet'].head(5))
print('\n\n\n')

Case Folding Result : 

0    @lisaamartatara3 @fadlizon yg dibangga banggai...
1    rt @rustamibrahim: polling: dari 4 gubernur di...
2    ojok ampe anies baswedan menikmati hsil proyek...
3    rt @ghanieierfan: pertemuan sangat mengharukan...
4    rt @sebastianteza: jangn nympek anies baswedan...
Name: tweet, dtype: object






In [16]:
import string 
import re #regex library

# import word_tokenize & FreqDist from NLTK
from nltk.tokenize import word_tokenize 
from nltk.probability import FreqDist

# ------ Tokenizing ---------

def remove_tweet_special(text):
    # remove tab, new line, ans back slice
    text = text.replace('\\t'," ").replace('\\n'," ").replace('\\u'," ").replace('\\',"")
    # remove non ASCII (emoticon, chinese word, .etc)
    text = text.encode('ascii', 'replace').decode('ascii')
    # remove mention, link, hashtag
    text = ' '.join(re.sub("([@#][A-Za-z0-9]+)|(\w+:\/\/\S+)"," ", text).split())
    # remove incomplete URL
    return text.replace("http://", " ").replace("https://", " ")
                
TWEET_DATA['tweet'] = TWEET_DATA['tweet'].apply(remove_tweet_special)

#remove number
def remove_number(text):
    return  re.sub(r"\d+", "", text)

TWEET_DATA['tweet'] = TWEET_DATA['tweet'].apply(remove_number)

#remove punctuation
def remove_punctuation(text):
    return text.translate(str.maketrans("","",string.punctuation))

TWEET_DATA['tweet'] = TWEET_DATA['tweet'].apply(remove_punctuation)

#remove whitespace leading & trailing
def remove_whitespace_LT(text):
    return text.strip()

TWEET_DATA['tweet'] = TWEET_DATA['tweet'].apply(remove_whitespace_LT)

#remove multiple whitespace into single whitespace
def remove_whitespace_multiple(text):
    return re.sub('\s+',' ',text)

TWEET_DATA['tweet'] = TWEET_DATA['tweet'].apply(remove_whitespace_multiple)

# remove single char
def remove_singl_char(text):
    return re.sub(r"\b[a-zA-Z]\b", "", text)

TWEET_DATA['tweet'] = TWEET_DATA['tweet'].apply(remove_singl_char)

# NLTK word rokenize 
def word_tokenize_wrapper(text):
    return word_tokenize(text)

TWEET_DATA['tweet_tokens'] = TWEET_DATA['tweet'].apply(word_tokenize_wrapper)

print('Tokenizing Result : \n') 
print(TWEET_DATA['tweet_tokens'].head())
print('\n\n\n')

Tokenizing Result : 

0    [yg, dibangga, banggain, sama, buzzer, cendana...
1    [rt, polling, dari, gubernur, di, jawa, ridwan...
2    [ojok, ampe, anies, baswedan, menikmati, hsil,...
3    [rt, pertemuan, sangat, mengharukan, dgn, ny, ...
4    [rt, jangn, nympek, anies, baswedan, menikmati...
Name: tweet_tokens, dtype: object






In [17]:
# NLTK calc frequency distribution
def freqDist_wrapper(text):
    return FreqDist(text)

TWEET_DATA['tweet_tokens_fdist'] = TWEET_DATA['tweet_tokens'].apply(freqDist_wrapper)

print('Frequency Tokens : \n') 
print(TWEET_DATA['tweet_tokens_fdist'].head().apply(lambda x : x.most_common()))

Frequency Tokens : 

0    [(yg, 2), (dibangga, 1), (banggain, 1), (sama,...
1    [(rt, 1), (polling, 1), (dari, 1), (gubernur, ...
2    [(ojok, 1), (ampe, 1), (anies, 1), (baswedan, ...
3    [(rt, 1), (pertemuan, 1), (sangat, 1), (mengha...
4    [(rt, 1), (jangn, 1), (nympek, 1), (anies, 1),...
Name: tweet_tokens_fdist, dtype: object


In [55]:
from nltk.corpus import stopwords

# ----------------------- get stopword from NLTK stopword -------------------------------
# get stopword indonesia
list_stopwords = stopwords.words('indonesian')
print(len(list_stopwords))

# ---------------------------- manualy add stopword  ------------------------------------
# append additional stopword
list_stopwords.extend(["yg", "dg", "rt", "dgn", "ny", "d", 'klo', 
                       'kalo', 'amp', 'biar', 'bikin', 'bilang', 
                       'gak', 'ga', 'krn', 'nya', 'nih', 'sih', 
                       'si', 'tau', 'tdk', 'tuh', 'utk', 'ya', 
                       'jd', 'jgn', 'sdh', 'aja', 'n', 't', 
                       'nyg', 'hehe', 'pen', 'u', 'nan', 'loh', 'rt',
                       '&amp', 'yah'])
len(list_stopwords)
# ----------------------- add stopword from txt file ------------------------------------
# read txt stopword using pandas
txt_stopword = pd.read_csv("stopwords.txt", names= ["stopwords"], header = None)

# convert stopword string to list & append additional stopword
list_stopwords.extend(txt_stopword["stopwords"][0].split(' '))
len(list_stopwords)
# ---------------------------------------------------------------------------------------

# convert list to dictionary
list_stopwords = set(list_stopwords)


#remove stopword pada list token
def stopwords_removal(words):
    return [word for word in words if word not in list_stopwords]

TWEET_DATA['tweet_tokens_WSW'] = TWEET_DATA['tweet_tokens'].apply(stopwords_removal) 


print(TWEET_DATA['tweet_tokens_WSW'].head())

0    [dibangga, banggain, buzzer, cendana, cuman, a...
1    [polling, gubernur, jawa, ridwan, kamil, anies...
2    [ojok, anies, baswedan, menikmati, proyek, mon...
3    [pertemuan, mengharukan, tsuneo, yamada, tua, ...
4    [anies, baswedan, menikmati, proyek, monas, pe...
Name: tweet_tokens_WSW, dtype: object


# Normalisasi

In [6]:
normalizad_word = pd.read_excel("normalisasi.xlsx")

normalizad_word_dict = {}

for index, row in normalizad_word.iterrows():
    if row[0] not in normalizad_word_dict:
        normalizad_word_dict[row[0]] = row[1] 

def normalized_term(document):
    return [normalizad_word_dict[term] if term in normalizad_word_dict else term for term in document]

TWEET_DATA['tweet_normalized'] = TWEET_DATA['tweet_tokens_WSW'].apply(normalized_term)

TWEET_DATA['tweet_normalized'].head(10)

0    [dibangga, banggakan, buzzer, cendana, cuman, ...
1    [polling, gubernur, jawa, ridwan, kamil, anies...
2    [jangan, sampai, anies, baswedan, menikmati, h...
3    [pertemuan, mengharukan, tsuneo, yamada, orang...
4    [jangan, nympek, anies, baswedan, menikmati, h...
5    [mencapai, anies, baswedan, menikmati, produk,...
6    [niat, cari, prestasi, ilegal, anies, baswedan...
7    [berbatas, anies, baswedan, menikmati, rakitan...
8    [jangan, sampai, anies, baswedan, menikmati, h...
9    [jangan, sampai, anies, baswedan, menikmati, h...
Name: tweet_normalized, dtype: object

In [7]:
# import Sastrawi package
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import swifter


# create stemmer
factory = StemmerFactory()
stemmer = factory.create_stemmer()

# stemmed
def stemmed_wrapper(term):
    return stemmer.stem(term)

term_dict = {}

for document in TWEET_DATA['tweet_normalized']:
    for term in document:
        if term not in term_dict:
            term_dict[term] = ' '
            
print(len(term_dict))
print("------------------------")

for term in term_dict:
    term_dict[term] = stemmed_wrapper(term)
    print(term,":" ,term_dict[term])
    
print(term_dict)
print("------------------------")


# apply stemmed term to dataframe
def get_stemmed_term(document):
    return [term_dict[term] for term in document]

TWEET_DATA['tweet_tokens_stemmed'] = TWEET_DATA['tweet_normalized'].swifter.apply(get_stemmed_term)
print(TWEET_DATA['tweet_tokens_stemmed'])

44203
------------------------
dibangga : bangga
banggakan : bangga
buzzer : buzzer
cendana : cendana
cuman : cuman
orang : orang
almarhum : almarhum
soeharto : soeharto
anies : anies
baswedan : baswedan
polling : polling
gubernur : gubernur
jawa : jawa
ridwan : ridwan
kamil : kamil
ganjar : ganjar
pranowo : pranowo
khofifah : khofifah
indar : indar
parawansa : parawansa
jangan : jangan
sampai : sampai
menikmati : nikmat
hasil : hasil
proyek : proyek
monas : monas
yang : yang
penuh : penuh
kejanggalannya : janggal
usutskandalmonasgate : usutskandalmonasgate
pertemuan : temu
mengharukan : haru
tsuneo : tsuneo
yamada : yamada
tua : tua
angkat : angkat
goodbener : goodbener
kuliah : kuliah
sophia : sophia
unive : unive
nympek : nympek
mencapai : capai
produk : produk
cetak : cetak
biru : biru
monumen : monumen
nasional : nasional
tumpat : tumpat
usutskandalmo : usutskandalmo
niat : niat
cari : cari
prestasi : prestasi
ilegal : ilegal
kuasai : kuasa
revitalisasi : revitalisasi
berbatas : b

lahat : lahat
penuhi : penuh
janji : janji
janjikan : janji
beliau : beliau
sumatera : sumatera
sumut : sumut
edy : edy
rahmayadi : rahmayadi
meninjau : tinjau
lokasi : lokasi
kecamatan : camat
barus : barus
tapanuli : tapanuli
tapteng : tapteng
kamis : kamis
edyrahmayadi : edyrahmayadi
gowatamapan : gowatamapan
dprddkijakarta : dprddkijakarta
muchlis : muchlis
legislator : legislator
pelajari : ajar
penanganan : tangan
gowa : gowa
harikerjaambyar : harikerjaambyar
har : har
bupati : bupati
batang : batang
cek : cek
bendung : bendung
glasing : glasing
siberuk : siberuk
jebol : jebol
menyebabkan : sebab
terendam : rendam
lumpur : lumpur
aktivitas : aktivitas
belajar : ajar
mengajar : ajar
terpaksa : paksa
dihentikan : henti
nagara : nagara
nusa : nusa
rimba : rimba
kebanjiran : banjir
anies lah : anies lah
jsp : jsp
febri : febri
kabar : kabar
sedap : sedap
kalimantan : kalimantan
pasalnya : pasal
akut : akut
terj : terj
ramalan : ramal
cuaca : cuaca
hujan : hujan
lebat : lebat
februari

jiwasraya : jiwasraya
pembentukan : bentuk
pansus : pansus
panja : panja
kpk : kpk
melemah : lemah
kantor : kantor
gagal : gagal
digeledah : geledah
flood : flood
gate : gate
stasiun : stasiun
mrt : mrt
tingginya : tinggi
pembelian : beli
hunian : huni
bertingkat : tingkat
mencari : cari
perkotaan : kota
pertimbangkan : timbang
umb : umb
kenalkan : kenal
sumur : sumur
resapan : resap
berbasis : bas
ecodrain : ecodrain
satunya : satu
ditemukan : temu
tandon : tandon
saluran : salur
kondisinya : kondisi
dangkal : dangkal
twips : twips
surabaya : surabaya
ayo : ayo
laporkan : lapor
twit : twit
me : me
nyinyir : nyinyir
laek : laek
doang : doang
pemerintah : perintah
cseluruh : cseluruh
diharap : harap
penyelamat : selamat
jakart : jakart
bro : bro
sis : sis
dukung : dukung
psi : psi
sejuta : juta
alasan : alas
dinyanyikan : nyanyi
berat : berat
menanggung : tanggung
ngukur : ngukur
diprediksi : prediksi
depannya : depan
memilih : pilih
apartemen : apartemen
pilihan : pilih
kelak : kelak
s

terintegrasi : integrasi
transportasi : transportasi
nama : nama
plaza : plaza
sel : sel
bangke : bangke
dibabat : babat
cowok : cowok
serba : serba
jon : jon
siapa : siapa
tags : tags
ceweknya : cewek
komentar : komentar
dihancurin : dihancurin
nglirik : nglirik
em : em
hoax : hoax
bersedih : sedih
lihat : lihat
babe : babe
bawa : bawa
poster : poster
jktbergerakturunkananies : jktbergerakturunkananies
skalinye : skalinye
bikin : bikin
biar : biar
mbemz : mbemz
mengakui : aku
jktberge : jktberge
menangis : menang
siwi : siwi
sidi : sidi
bantah : bantah
gundik : gundik
bos : bos
garuda : garuda
asyiiik : asyiiik
ramai : ramai
dcf : dcf
dfff : dfff
ngaku : ngaku
ngaji : ngaji
filsafat : filsafat
ngajifilsafat : ngajifilsafat
masjidjendralsudirman : masjidjendralsudirman
fahruddinfaiz : fahruddinfaiz
yogyakarta : yogyakarta
perdamaian : damai
twitter : twitter
instagram : instagram
diam : diam
kah : kah
becus : becus
rakyattolakrevitalisasimonas : rakyattolakrevitalisasimonas
aniesgabisa

lutfi : lutfi
dipaksa : paksa
mengaku : aku
melempar : lempar
batu : batu
conquestofconstantinople : conquestofconstantinople
globalcampaign : globalcampaign
yth : yth
btw : btw
triliun : triliun
biopori : biopori
ditiap : tiap
minimal : minimal
keren : keren
ddf : ddf
dde : dde
banjirjakarta : banjirjakarta
jum : jum
at : at
mubarak : mubarak
senyummu : senyum
menghadirkan : hadir
nyaman : nyaman
damai : damai
jakarteimlekan : jakarteimlekan
pemimpinnye : pemimpinnye
gong : gong
xi : xi
fa : fa
cai : cai
senangwargenye : senangwargenye
gembira : gembira
pengunjungnye : pengunjungnye
duduk : duduk
berdiskusi : diskus
santai : santai
petinggi : petinggi
ulama : ulama
umaro : umaro
rj : rj
lino : lino
tersangka : sangka
korupsi : korupsi
pengadaan : ada
pelindo : pelindo
ditahan : tahan
tunggu : tunggu
modar : modar
demokrat : demokrat
halu : halu
om : om
bermuara : muara
mul : mul
panglima : panglima
tni : tni
beyeh : beyeh
berbohong : bohong
majubersama : majubersama
berjuang : juang
l

...

breakthrough : breakthrough
saying : saying
name : name
blowned : blowned
experiencing : experiencing
nursery : nursery
pla : pla
indonesiarising : indonesiarising
bakat : bakat
stage : stage
gbk : gbk
wohoo : wohoo
vanue : vanue
ccreative : ccreative
cpitch : cpitch
ammho : ammho
creators : creators
ground : ground
dance : dance
hellodangdut : hellodangdut
excitement : excitement
continues : continues
dayu : dayu
dara : dara
femaledistruptor : femaledistruptor
nar : nar
actor : actor
cperempuan : cperempuan
crowd : crowd
cooming : cooming
irfan : irfan
cmovie : cmovie
script : script
having : having
nostalgila : nostalgila
ultra : ultra
supporting : supporting
stages : stages
packed : packed
glorious : glorious
installations : installations
glimpse : glimpse
buzz : buzz
farewell : farewell
matanajwahidupmatikpk : matanajwahidupmatikpk
capim : capim
busyro : busyro
muqoddas : muqoddas
akuntabel : akuntabel
capimnya : capimnya
menginspirasimu : inspirasi
mengupas : kupas
biografi : biog

peacock : peacock
library : library
dikurasi : rasi
mendongeng : dongeng
pendongeng : dongeng
klubbukunarasi : klubbukunarasi
onderdil : onderdil
matanajwademibisnisnegara : matanajwademibisnisnegara
sahabatnya : sahabat
penunjukkan : tunjuk
polisnya : polis
celia : celia
bromance : bromance
cast : cast
jumanji : jumanji
junket : junket
wasathiyyah : wasathiyyah
menganut : anut
eggnoid : eggnoid
cintaan : cinta
terusik : usik
yeong : yeong
ckim : ckim
perundungan : rundung
hatchie : hatchie
frankie : frankie
cosmos : cosmos
zaskia : zaskia
adya : adya
mecca : mecca
webtoon : webtoon
redcat : redcat
opera : opera
operavanjava : operavanjava
transmedia : transmedia
jenaka : jenaka
casugay : casugay
peselancar : selancar
medal : medal
redy : redy
pengagas : agas
binaannya : bina
salampancasila : salampancasila
nose : nose
matanajwajenakadinegeriopera : matanajwajenakadinegeriopera
cagur : cagur
pemainnya : main
cdungu : cdungu
lekat : lekat
ketenaran : tenar
blur : blur
mendulang : dulang

messaging : messaging
favoritmu : favorit
pembekalan : bekal
menindaklanjutinya : menindaklanjutinya
pengkajian : kaji
gta : gta
braile : braile
dimodifikasi : modifikasi
disusulkan : susul
tunanetra : tunanetra
pertuni : tun
braille : braille
photofunia : photofunia
summary : summary
bahasannya : bahas
beropini : opini
subyektifnya : subyektif
independent : independent
relay : relay
ditimpa : timpa
txt : txt
mersin : mersin
antalya : antalya
acungkan : acung
lamongan : lamongan
akutudahmalas : akutudahmalas
ekuinoks : ekuinoks
paladin : paladin
paf : paf
igf : igf
distribusinya : distribusi
irah : irah
blabla : blabla
cdebat : cdebat
chechnya : chechnya
ortumu : ortumu
rinz : rinz
dituliskan : tulis
terjemahan : terjemah
hebatdiki : hebatdiki
dipindai : pindai
korups : korups
fackchecking : fackchecking
gakkumdu : gakkumdu
ketidakberpihakan : ketidakberpihakan
mencuitkan : cuit
pengarang : arang
undercover : undercover
mudanya : muda
unsoed : unsoed
antonio : antonio
banerra : banerra

C:\Users\yunus\Anaconda3\envs\ROOT\lib\site-packages\tqdm\std.py:666: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel



0        [bangga, bangga, buzzer, cendana, cuman, orang...
1        [polling, gubernur, jawa, ridwan, kamil, anies...
2        [jangan, sampai, anies, baswedan, nikmat, hasi...
3        [temu, haru, tsuneo, yamada, orang, tua, angka...
4        [jangan, nympek, anies, baswedan, nikmat, hasi...
                               ...                        
50641    [afaik, pardon, presiden, bentuk, amnesti, gra...
50642    [didu, sayang, luhut, paham, otak, uang, uang,...
50643    [rapat, kemenlu, anggota, dpr, ingat, recok, m...
50644    [bekas, beda, galaksi, sana, dengar, dengar, p...
50645    [mending, rakyat, ramai, ancam, luhut, negara,...
Name: tweet_tokens_stemmed, Length: 50646, dtype: object


In [8]:
TWEET_DATA.to_csv("Text_Preprocessing.csv")

In [9]:
TWEET_DATA.to_excel("Text_Preprocessing.xlsx")

In [10]:
TWEET_DATA.to_hdf("Text_Preprocessing.h5", 'tweet', mode='w')

C:\Users\yunus\Anaconda3\envs\ROOT\lib\site-packages\pandas\core\generic.py:2505: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block1_values] [items->Index(['tweet', 'tweet_tokens', 'tweet_tokens_fdist', 'tweet_tokens_WSW',
       'tweet_normalized', 'tweet_tokens_stemmed'],
      dtype='object')]

  encoding=encoding,
